<a href="https://colab.research.google.com/github/dirknbr/privacy/blob/main/privacy_noise_debias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Privacy: retrieving the unbiased effect after adding privacy noise to data

Disclaimer: This is purely personal research and not endorsed by my employer.

Imagine you want to make some data more private by adding some noise to each data point. You then want to estimate the effect of x on y in a simple regression model. But the noise (like measurement error) creates a bias in your linear effect, it shrinks it to zero. 

However, if we know some aggregate stats about X and the noise we can unbias the effect again. This means we have protected the data but also can still infer the "true" effect.

The unbiasing step is based on this [1992 paper](https://epiresearch.org/wp-content/uploads/2014/07/AJE1992Rosner.pdf)





In [1]:
import numpy as np
from sklearn import linear_model
# https://www.statsmodels.org/stable/gettingstarted.html
import statsmodels.api as sm


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# the true simulated data, true beta is [0, -1]
np.random.seed(33)
N = 800
X = np.random.normal(10, 1.1, size=(N, 2))
y = 5 - X[:, 1] + np.random.normal(size=N)


In [3]:
# some functions we need, we use laplace noise, we also return some covariance

def add_noise_matrix(X, sd=.3):
  noise = np.random.laplace(0, sd, size=X.shape)
  return X + noise, np.cov(X.T), np.cov(noise.T)

def add_intc(X):
  n, k = X.shape
  X2 = np.ones((n, k + 1))
  X2[:, 1:] = X
  return X2


In [4]:
Xno, cov_X, cov_noise = add_noise_matrix(X)
print(cov_noise)


[[ 0.17409356 -0.01073539]
 [-0.01073539  0.18182556]]


In [5]:
# some stats, the mean is unchanged
print('mean')
print(X.mean(axis=0))
print(Xno.mean(axis=0))

print('std')
print(X.std(axis=0))
print(Xno.std(axis=0))


mean
[10.03795965  9.98199436]
[10.04930025  9.98839855]
std
[1.10880734 1.05440384]
[1.17761255 1.14254982]


In [6]:
# we run the original data and the biased (private) data

m1 = sm.OLS(y, add_intc(X))
m3 = sm.OLS(y, add_intc(Xno))

res1 = m1.fit()
res3 = m3.fit()

print(res1.summary())
print(res3.summary())

# we see that the Xno model has some bias in beta
# also take some note of the standard errors

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.536
Model:                            OLS   Adj. R-squared:                  0.535
Method:                 Least Squares   F-statistic:                     459.8
Date:                Mon, 28 Jun 2021   Prob (F-statistic):          1.67e-133
Time:                        15:33:50   Log-Likelihood:                -1126.7
No. Observations:                 800   AIC:                             2259.
Df Residuals:                     797   BIC:                             2274.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.1791      0.459     11.287      0.0

In [7]:
# we use 200 bootstraps to get an interval around our new estimate

S_X = cov_X
S = cov_noise

# b_hat = res3.params[1:]

# bootstrappping
b_samples = np.zeros((200, 2))

for i in range(200):
  sample = np.random.choice(N, N, replace=True)
  Xno_sel = Xno[sample, :]
  y_sel = y[sample]
  m3 = sm.OLS(y_sel, add_intc(Xno_sel))
  res3 = m3.fit()
  b_hat = res3.params[1:]

  S_Z = np.cov(Xno_sel.T)

  b_fixed = b_hat.dot(np.identity(2) + S.dot(np.linalg.inv(S_X)))
  # b_fixed2 = b_hat.dot(np.linalg.inv(S_X.dot(np.linalg.inv(S_Z))))
  b_samples[i, :] = b_fixed

print('bootstrapped')
print(b_samples.mean(axis=0))
print(b_samples.std(axis=0))


bootstrapped
[-0.00607441 -0.9910633 ]
[0.03471681 0.03663509]


The unbiased estimate of beta is

$$\beta = \hat{\beta}(I + \Sigma\Sigma_X^{-1})$$

where $\hat{\beta}$ is the biased estimate, $\Sigma$ is the covariance of the noise, $\Sigma_X$ is the covariance of X (before adding noise).

So if we have a biased X but we also know the original covariance, we can fix the estimates. 